**Clustream Algorithm**

In [0]:
import numpy as np
from sklearn import datasets, metrics


#Reading data from a csv file
from pandas import read_csv
data=read_csv(filepath_or_buffer="newdiabetes1.csv", header='infer')
print(type(data))
newdata=data.values
print("Type:",type(newdata))
print("Shape:",newdata.shape)

<class 'pandas.core.frame.DataFrame'>
Type: <class 'numpy.ndarray'>
Shape: (156, 9)


Perform Scalling of the datasets using Standard Scaler 

In [0]:
#scale the data using standard scaler
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(newdata)
newdata=scaler.transform(newdata)

# Perform necessary initialization of the data

In [0]:
# To display number of the values in the datasets
no_row= newdata.shape[0]
print("Total no of data in datasets =",no_row)

chunk_size=6          
chunk_limit=5*chunk_size 
k=2                        #  no of medians 

# Create single level matrix to store score of various levels of clusters . I assume that at max 10 levels will occurs 
level=np.zeros([10,100, 2],dtype=int)


#create level_count matrix to store the count of weights in each chunk
level_count=np.zeros(10,dtype=int)


Total no of data in datasets = 156


# User defined function to calculate number of clusters from Single Chunk

In [0]:
from sklearn.metrics.pairwise import pairwise_distances
from numpy.random import choice
from numpy.random import seed
from pyclustering.cluster.kmedoids import kmedoids   
from pyclustering.cluster.kmedians import kmedians


def  mycluster(data1,k=2):

    # Code for Random initialization of the Median for single chunk
    seed(1)
    samples=choice(len(data1),size=k,replace=False)
    initial_medians=data1[samples,:]
    # print("Initial Centroids",initial_medians)
    
    
    #code for the k-median cluster of single chunk
    kmedians_instance = kmedians(data1, initial_medians)
    # Run cluster analysis and obtain results.
    kmedians_instance.process()
    clusters1 = kmedians_instance.get_clusters()
    return clusters1 
    

# Core logic of Stream algorithm. Find the cluseters of the data points and store & add the weightage of the clusters in the different level

In [0]:
#pip install pyclustering


from sklearn.metrics.pairwise import pairwise_distances
from numpy.random import choice
from numpy.random import seed
from pyclustering.cluster.kmedians import kmedians


#print(level)

chunk_no=0

# call the forloop to iterate over all the data with the increament of chunk_size
for i in range(1,no_row,chunk_size):
    
    # store single chunk in data1
    data1=newdata[i:i+chunk_size]
    # print("data of chunk" ,data1)
    #print(data1.shape)  
    
    
    # Call myCluster function to find the cluster for single data clusters
    clusters=mycluster(data1,2)
    
    #store the count of each cluster in level1 matrix 
    cluslength=len(clusters)
    level_number=0
    clusterno=0
    
    # Update the cluster weightage in the first level for Single data cluster 
    for i in clusters:
        level[0][chunk_no][clusterno]=len(i)
       # level[level_number][chunk_no][clusterno]=len(i)
        clusterno=clusterno+1
  
    chunk_no=chunk_no+1
   
    level_count[0]=level_count[0] + 1
    first_level_count=level_count[0]
     
    # Code to check if dataframe of any level has exceed the size or not
    if ((first_level_count * k) % chunk_size) == 0: 
        current_level=level_number  
        x=1 
        
        # Iterate while loop continuously until we update necessary level's data
        while x==1:
            
            next_level=current_level + 1
            next_level_chunk_no=level_count[next_level]
            
            # count the weight of different clusters in the level matrix            
            for p in range(k):
                 
                l=level_count[current_level]
                k1_count=level[current_level][l-1][p]+level[current_level][l-2][p]+level[current_level][l-3][p]
      
                level[next_level][next_level_chunk_no][p]=k1_count
            
            level_count[next_level]= next_level_chunk_no + 1
            
            if ((level_count[next_level])* k) % chunk_size == 0  :
                current_level=next_level
                next_level=next_level +1
                
            else :
                 x=2

    
# Print the number of cluster weightage per data chunk at each level
for i in range(10):
    if level_count[i] > 0 : 
        print()
        print("Level no :",i)
        l=level_count[i]
        for j in range(l):
            print("Chunk No ",j," :",level[i][j])

print()            
# Print number of dataframes at individual level
for j in range(10):
    if level_count[j] > 0 :
        max_level=j
        print("No of dataframes at level ",j,"=",level_count[j])

print("Last Highest Level =",max_level)


# create 2 dimensional matrix to store final cluster weightage count
final_cluster=np.zeros([1,k],dtype=int)

# Check if maximul level has number of elements less than chunk size then add all the elements for final summation
if (level_count[max_level] * k) % chunk_size != 0  :
    
    # Find the final level
    m=level_count[max_level] % chunk_size 
        
    # Iterate over all the k-clusters of maximuk level to find sum
    for i in range(k):
       # print("i=",i)
       # print("length of i =",len(i))
        for j in range(m):
            final_cluster[0][i]=final_cluster[0][i]+level[max_level][j][i]


# print final cluster weightage of the data
print()
print("FINAL CLUSTER WEIGHTAGE : ",final_cluster)
        


Level no : 0
Chunk No  0  : [4 2]
Chunk No  1  : [4 2]
Chunk No  2  : [5 1]
Chunk No  3  : [4 2]
Chunk No  4  : [1 5]
Chunk No  5  : [4 2]
Chunk No  6  : [2 4]
Chunk No  7  : [1 5]
Chunk No  8  : [4 2]
Chunk No  9  : [4 2]
Chunk No  10  : [3 3]
Chunk No  11  : [4 2]
Chunk No  12  : [1 5]
Chunk No  13  : [1 5]
Chunk No  14  : [4 2]
Chunk No  15  : [1 5]
Chunk No  16  : [2 4]
Chunk No  17  : [4 2]
Chunk No  18  : [1 5]
Chunk No  19  : [4 2]
Chunk No  20  : [1 5]
Chunk No  21  : [2 4]
Chunk No  22  : [2 4]
Chunk No  23  : [2 4]
Chunk No  24  : [5 1]
Chunk No  25  : [1 4]

Level no : 1
Chunk No  0  : [13  5]
Chunk No  1  : [9 9]
Chunk No  2  : [ 7 11]
Chunk No  3  : [11  7]
Chunk No  4  : [ 6 12]
Chunk No  5  : [ 7 11]
Chunk No  6  : [ 6 12]
Chunk No  7  : [ 6 12]

Level no : 2
Chunk No  0  : [29 25]
Chunk No  1  : [24 30]

No of dataframes at level  0 = 26
No of dataframes at level  1 = 8
No of dataframes at level  2 = 2
Last Highest Level = 2

FINAL CLUSTER WEIGHTAGE :  [[53 55]]
